In [1]:
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import pandas as pd 
import joblib

In [2]:
filePath = "data/data.csv" 
spotify_df = pd.read_csv(filePath)
spotify_df = spotify_df.set_index("id")
spotify_df.head(5)

,acousticness,artists,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
id,,,,,,,,,,,,,,,,,,
0cS0A1fUEUd1EW3FcF8AEI,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
0hbkKFIJm7Z05H8Zl9w30f,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
11m7laMUgmOKqI3oYzuhne,0.993000,['Mamie Smith'],0.647,163827,0.186,0,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
19Lc5SfJJ5O1oaxY0fpwfh,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
2hJjbsLCytGsnAHfdsLejp,0.295000,['Mixe'],0.704,165224,0.707,1,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920


In [3]:
spotify_df_features = spotify_df[["acousticness", "danceability", "duration_ms", "energy", "instrumentalness", "key", "liveness", "loudness", "speechiness", "tempo", "valence"]]

In [4]:
scaler = StandardScaler()
spotify_scaler = scaler.fit(spotify_df_features) # save later 
spotify_df_scaled = spotify_scaler.transform(spotify_df_features)

In [5]:
run_analysis = False 

In [6]:
kmeans_kwargs = {
    "init": "k-means++",
    "n_init": 10,
    "max_iter": 300,
    "random_state": 42
}

if run_analysis: 
    n_cluster_start = 2
    n_cluster_end = 25 

    sse = []
    silhouette_coefficients = []
    for i in range(n_cluster_start, n_cluster_end): 
        kmeans = KMeans(n_clusters=i, **kmeans_kwargs)
        kmeans.fit(spotify_df_scaled)
        sse.append(kmeans.inertia_)
        score = silhouette_score(spotify_df_scaled, kmeans.labels_) 
        silhouette_coefficients.append(score)

In [7]:
if run_analysis:
    plt.plot(range(n_cluster_start, n_cluster_end), sse)
    plt.xticks(range(n_cluster_start, n_cluster_end))
    plt.xlabel("Number of Clusters")
    plt.ylabel("SSE")
    plt.show()

In [8]:
if run_analysis:
    kl = KneeLocator(range(n_cluster_start,n_cluster_end), sse, curve="convex", direction="decreasing")
    kl.elbow

In [9]:
if run_analysis:
    plt.plot(range(n_cluster_start, n_cluster_end), silhouette_coefficients)
    plt.xticks(range(n_cluster_start, n_cluster_end))
    plt.xlabel("Number of Clusters")
    plt.ylabel("Silhouette Coefficient")
    plt.show()

In [10]:
# create model from elbow 
kmeans = KMeans(n_clusters=25, **kmeans_kwargs)
kmeans.fit(spotify_df_scaled)

KMeans(n_clusters=25, random_state=42)

In [11]:
kmeans.cluster_centers_

array([[-1.07656218e+00, -9.28081457e-01,  5.55762824e-02,
         1.11514250e+00, -4.76923179e-01, -2.91540229e-01,
        -1.06824378e-01,  8.55899894e-01, -7.42965374e-02,
         1.67657852e+00, -2.45419643e-01],
       [ 4.14023761e-01,  2.17798024e-01, -2.85986800e-01,
         1.77735159e-01, -4.69188481e-01, -4.23632306e-02,
         1.50860169e+00,  3.46253756e-01, -1.01726763e-01,
        -2.61065267e-01,  8.09035760e-01],
       [ 1.05299659e+00, -7.66023378e-01, -2.47247301e-02,
        -1.06794115e+00,  1.89254006e+00, -1.93479451e-01,
        -3.43991846e-01, -1.22514424e+00, -2.73524271e-01,
         8.93824390e-01, -7.76090922e-01],
       [-1.10631707e+00, -7.19025994e-02,  1.15040783e-02,
         1.23438962e+00, -4.88123103e-01, -9.15462556e-01,
         2.98713945e-02,  9.84291944e-01, -1.41254611e-01,
        -1.10239290e-01, -1.37429775e-01],
       [-9.05715343e-01,  2.65061297e-01, -1.33599479e-01,
         1.04024870e+00, -4.94666593e-01,  8.59770923e-01,
  

In [12]:
labels = kmeans.predict(spotify_df_scaled)

In [13]:
spotify_df["label"] = labels
spotify_df

,acousticness,artists,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,label
id,,,,,,,,,,,,,,,,,,,
0cS0A1fUEUd1EW3FcF8AEI,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920,13
0hbkKFIJm7Z05H8Zl9w30f,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920,6
11m7laMUgmOKqI3oYzuhne,0.993000,['Mamie Smith'],0.647,163827,0.186,0,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920,21
19Lc5SfJJ5O1oaxY0fpwfh,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920,10
2hJjbsLCytGsnAHfdsLejp,0.295000,['Mixe'],0.704,165224,0.707,1,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46LhBf6TvYjZU2SMvGZAbn,0.009170,"['DJ Combo', 'Sander-7', 'Tony T']",0.792,147615,0.866,0,0.000060,6,0.1780,-5.089,0,The One,0,2020-12-25,0.0356,125.972,0.1860,2020,15
7tue2Wemjd0FZzRtDrQFZd,0.795000,['Alessia Cara'],0.429,144720,0.211,0,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.2280,2021,22
48Qj61hOdYmUCFJbpQ29Ob,0.806000,['Roger Fly'],0.671,218147,0.589,0,0.920000,4,0.1130,-12.393,0,Together,0,2020-12-09,0.0282,108.058,0.7140,2020,19


In [14]:
# save labeled df to csv 
spotify_df.to_csv("data/spotify_data_labeled.csv")

In [16]:
# export scaler 
joblib.dump(spotify_scaler, "model/scaler.joblib")
# export model
joblib.dump(kmeans, "model/kmeans.joblib")

['model/kmeans.joblib']